In [2]:
#imports
import json
import os
import warnings
from datetime import date, datetime, timedelta

import pandas as pd
import requests
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")

MORNING_LEAD_LIST_ID = 214864
MORNING_BROADCAST_ID = 82319
EVENING_LEAD_LIST_ID = 214867
EVENING_BROADCAST_ID = 82321

## auth token 
# from login import auth_token

#db information
user1 = "SFPL_Connect"
password = "$%n5bF33%X"
db = "Sonata_Connect"
server = "172.17.130.216"
engine = create_engine(f"mssql+pyodbc://{user1}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server")

def get_authorization():
    """ Get auth token """
    username = os.environ['PAYMEE_SONATA_TATA_USERNAME']
    password = os.environ['PAYMEE_SONATA_TATA_PASSWORD']
    print(username,password)
    url = "https://api-smartflo.tatateleservices.com/v1/auth/login"

    payload = {
        "email": username,
        "password": password
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()['access_token']
    else :
        return None
    
auth_token = get_authorization()


TACN2479 SonataFin123@@


In [3]:

today_date = str(date.today())
today_date = datetime.strptime(today_date, '%Y-%m-%d').strftime('%Y-%m-%d')
sql_query = f""" SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF """
hierarchy_df=pd.read_sql_query(sql_query,engine)

In [4]:
bro_df = hierarchy_df[hierarchy_df['RoleId'] == 55][['UserID', 'UserName']]

In [5]:
bro_df

,UserID,UserName
254,4829,ANKIT
370,3353,RAHUL KUMAR
372,25715,DHEERAJ KUMAR
373,27274,Saurabh Kushwaha
374,31073,NEERAJ KUMAR
...,...,...
3010,31527,ANNU KUSHWAHA
3011,31751,SAKSHI CHOUDHARY
3013,27196,Mithilesh Kumar Paswan
3015,31803,SHANU KUMAR


In [6]:
sql_query = f"""SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF"""
df=pd.read_sql_query(sql_query,engine)
df = df.replace(['Prayagraj Division','Lucknow Division'],['Prayagraj', 'Lucknow'])

df = df.replace(['Prayagraj Division','Lucknow Division'],['Prayagraj', 'Lucknow'])
#WE CREATE A NEW USER HIERARCHY TABLE
div_df = df[(df['RoleId']==34)&(df['RoleName']=='DIVISION HEAD')][['U_BUID','UserID','UserName','buname']]
print(len(div_df))
div_df.rename(columns={'U_BUID':'Div_BUID','buname':'DivName','UserID':'Div_UserID','UserName':'Div_UserName'}, inplace=True)
reg_df = df[(df['RoleId']==35)&(df['RoleName']=='REGION HEAD')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
print(len(reg_df))
reg_df.rename(columns={'U_BUID':'Region_BUID','ReportingBUId':'Region_ReportingBUId','buname':'RegionName','UserID':'Region_UserID','UserName':'Region_UserName'}, inplace=True)
# hub_df = df[(df['RoleId']==44)&(df['RoleName']=='Credit Committee')][['U_BUID','UserID','UserName','buname','ReportingBUId']]
hub_df = df[(df['RoleId'].isin([36,44,42]))&(df['RoleName'].isin(['Credit Committee','Credit Officer','HUB HEAD']))][['U_BUID','UserID','UserName','buname','ReportingBUId','RoleId']]
#We will classify HUB,CRC,CRO on the basis of RID
hub_df.rename(columns={'U_BUID':'Hub_BUID','ReportingBUId':'Hub_ReportingBUId','buname':'HubName','UserID':'Hub_UserID','UserName':'Hub_UserName','RoleId':'RID'}, inplace=True)
bm_df = df[(df['RoleId']==13)&(df['RoleName']=='Branch Manager')][['U_BUID','buname','UserID','UserName','ReportingBUId']]
bm_df.rename(columns={'U_BUID':'BM_BUID','ReportingBUId':'BM_ReportingBUId','buname':'BranchName','UserID':'BM_UserID','UserName':'BM_UserName'}, inplace=True)
bro_df = df[(df['RoleId']==55)&(df['RoleName']=='BRO')][['U_BUID','UserID','UserName','buname']]
bro_df.rename(columns={'U_BUID':'BRO_BUID','UserID':'BRO_UserID','UserName':'BRO_UserName','buname':'BRO_buname'}, inplace=True)
M1 = pd.merge(bro_df,bm_df,left_on='BRO_BUID',right_on='BM_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M2 = pd.merge(M1,hub_df,left_on='BM_ReportingBUId',right_on='Hub_BUID',how='left').drop_duplicates(subset='BRO_UserID')
M3 = pd.merge(M2,reg_df,left_on='Hub_ReportingBUId',right_on='Region_BUID',how='left').drop_duplicates(subset='BRO_UserID')
dF = pd.merge(M3,div_df,left_on='Region_ReportingBUId',right_on='Div_BUID',how='left').drop_duplicates(subset='BRO_UserID')

11
45


In [8]:

cols_to_fill = ['BM_BUID','BM_UserID','BM_ReportingBUId','Hub_BUID','Hub_UserID','Hub_ReportingBUId','RID', 'Region_BUID','Region_UserID','Region_ReportingBUId','Div_BUID','Div_UserID']
dF[cols_to_fill] = dF[cols_to_fill].fillna(0).astype('int', errors='ignore')

In [9]:
dF

,BRO_BUID,BRO_UserID,BRO_UserName,BRO_buname,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,...,RID,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
0,591,4829,ANKIT,Badaun,0,NaN,0,NaN,0,0,...,0,0,0,NaN,NaN,0,0,0,NaN,NaN
1,2,3353,RAHUL KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,...,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
2,2,25715,DHEERAJ KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,...,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
3,2,27274,Saurabh Kushwaha,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,...,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
4,2,31073,NEERAJ KUMAR,Sirathu,2,Sirathu,4797,PINTU KUMAR YADAV,545,545,...,36,2180,18044,KAMLESH YADAV,Kaushambi,650,650,9739,LAKSHMAN SINGH,Prayagraj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,2324,31527,ANNU KUSHWAHA,Aadhartal,2324,Aadhartal,26898,Ruby Dahiya,562,562,...,36,1007,1174,shama parveen,Jabalpur,652,652,15190,JITENDRA YADAV,Jabalpur
2088,2324,31751,SAKSHI CHOUDHARY,Aadhartal,2324,Aadhartal,26898,Ruby Dahiya,562,562,...,36,1007,1174,shama parveen,Jabalpur,652,652,15190,JITENDRA YADAV,Jabalpur
2089,2325,27196,Mithilesh Kumar Paswan,Nirmali,2325,Nirmali,25630,Dharmveer Kumar,2104,2104,...,36,2198,10848,SHYAM KISHOR MISHRA,Darbhanga,653,653,10849,KESHRAJ YADAV,Muzaffarpur
2090,2326,31803,SHANU KUMAR,Chandwa,2326,Chandwa,27375,Rahul Raj,2206,2206,...,36,2201,17841,MURARI MAURYA,Hazaribagh RO,2200,2200,22544,Shiv Kumar Singh,Ranchi DO


,UserID,UserName,CallingNumber,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count


In [10]:
today_date = datetime.strptime(str(date.today()), '%Y-%m-%d').strftime('%Y-%m-%d')
sql_query = f""" SET NOCOUNT ON;EXEC [{db}].[dbo].[SP_UserHierarchy_Dynamic_07Jan23] @userid = 1;SET NOCOUNT OFF """
hierarchy_df=pd.read_sql_query(sql_query,engine)
hierarchy_df = hierarchy_df[hierarchy_df['RoleId']==55]
hierarchy_df = hierarchy_df[['UserID','UserName']]
userid_tuple = tuple(hierarchy_df['UserID'].unique().tolist())
sql_query = f""" SELECT CallingNumber,UserID FROM Sonata_Connect.dbo.accounts_calling_number_list
    WHERE UserID in {userid_tuple} """
contact_df = pd.read_sql_query(sql_query,engine)
main_df = pd.merge(hierarchy_df,contact_df,on='UserID',how='right')
# sql_query = f""" SELECT TOP 10 assigned_to,status,deadline_date FROM Sonata_Connect.dbo.accounts_task_details
#     WHERE assigned_to in {userid_tuple} and deadline_date = {today_date} """
sql_query = f""" SELECT assigned_to,status,deadline_date,category FROM Sonata_Connect.dbo.accounts_task_details
    WHERE deadline_date = '{today_date}' and category in ('3','5','7','10')"""

task_df = pd.read_sql_query(sql_query,engine)
task_df = task_df[(task_df['status'] == 'Pending')|(task_df['status'] == 'Re-Schedule')]
pending_task_df = task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
pending_task_df = pending_task_df.rename(columns={'deadline_date':'Pending_Count'})

print("Length of pending taks : ", len(pending_task_df))

collection_task_df = task_df[task_df['category']=='3']
collection_task_df = collection_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
collection_task_df = collection_task_df.rename(columns={'deadline_date':'collection_count'})

proposal_task_df = task_df[task_df['category']=='5']
proposal_task_df = proposal_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
proposal_task_df = proposal_task_df.rename(columns={'deadline_date':'proposal_count'})

retention_task_df = task_df[task_df['category']=='7']
retention_task_df = retention_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
retention_task_df = retention_task_df.rename(columns={'deadline_date':'retention_count'})

wrong_number_task_df = task_df[task_df['category']=='10']
wrong_number_task_df = wrong_number_task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
wrong_number_task_df = wrong_number_task_df.rename(columns={'deadline_date':'wrong_number_count'})

task_df = task_df[(task_df['status'] == 'Completed')]
completed_task_df = task_df.groupby('assigned_to')['deadline_date'].count().reset_index()
completed_task_df = completed_task_df.rename(columns={'deadline_date':'Completed_Count'})

main_df = pd.merge(main_df,pending_task_df,left_on='UserID',right_on='assigned_to',how='right')
main_df = pd.merge(main_df,collection_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,proposal_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,retention_task_df,left_on='UserID',right_on='assigned_to',how='left')
main_df = pd.merge(main_df,wrong_number_task_df,left_on='UserID',right_on='assigned_to',how='left')


main_df = main_df.drop(columns=['assigned_to_y', 'assigned_to_x'], axis=1).dropna(subset=['UserID']).fillna(0).reset_index(drop=True)

main_df['collection_count'] = main_df['collection_count'].astype(int, errors='ignore')
main_df['proposal_count'] = main_df['proposal_count'].astype(int, errors='ignore')
main_df['retention_count'] = main_df['retention_count'].astype(int, errors='ignore')
main_df['wrong_number_count'] = main_df['wrong_number_count'].astype(int, errors='ignore')

main_df['UserID'] = main_df['UserID'].astype(int, errors='ignore')
main_df['UserName'] = main_df['UserName'].apply(lambda x : x.title())

Length of pending taks :  561


In [13]:
main_df = main_df.drop('assigned_to', axis=1)

In [14]:
main_df

,UserID,UserName,CallingNumber,Pending_Count,collection_count,proposal_count,retention_count,wrong_number_count
0,1478,Ram Roop,9076605730,1,0,0,1,0
1,1709,Ballu,8960154523,2,0,0,2,0
2,1934,Vijay Kumar,7897997362,1,0,0,1,0
3,2107,Chiraunji Lal,7318080248,1,0,0,1,0
4,2916,Vijay Kumar,7080200377,1,0,0,1,0
...,...,...,...,...,...,...,...,...
502,31779,Prity Kumari,9142222360,3,3,0,0,0
503,31810,Pankaj Kumar,9065230039,1,1,0,0,0
504,31827,Sumit Kumar,8192892088,7,7,0,0,0
505,31836,Mukesh Verma,6261403746,2,2,0,0,0


In [18]:
dF[dF['BRO_UserID'] == 31810]

,BRO_BUID,BRO_UserID,BRO_UserName,BRO_buname,BM_BUID,BranchName,BM_UserID,BM_UserName,BM_ReportingBUId,Hub_BUID,...,RID,Region_BUID,Region_UserID,Region_UserName,RegionName,Region_ReportingBUId,Div_BUID,Div_UserID,Div_UserName,DivName
1078,357,31810,PANKAJ KUMAR,Mashrakh,357,Mashrakh,4964,SUMIT KUMAR,605,605,...,36,1012,24776,RAJNAND MAURYA,Muzaffarpur,653,653,10849,KESHRAJ YADAV,Muzaffarpur


In [24]:
today_df = pd.merge(left=main_df, right=dF, left_on='UserID', right_on='BRO_UserID', how='left')


In [25]:
len(today_df['CallingNumber'].unique())

507